In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)
from openpyxl import Workbook, load_workbook
import os
import io

# Function to calculate classification metrics
def calculate_classification_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    gini = 2 * auc - 1
    confusion = confusion_matrix(y_true, y_pred)

    metrics = {
        'Metric': ['Precision', 'Recall', 'F1-Score', 'AUC ROC Score', 'GINI'],
        'Value': [precision, recall, f1, auc, gini]
    }
    return metrics, confusion

# Function to save classification results to Excel
def save_classification_results_to_excel(directory, model_name, sheet_name, results, confusion):
    file_name = os.path.join(directory, f'{model_name}_classification_results.xlsx')
    try:
        workbook = load_workbook(file_name)
    except FileNotFoundError:
        workbook = Workbook()
        if 'Sheet' in workbook.sheetnames:
            del workbook['Sheet']

    if sheet_name not in workbook.sheetnames:
        workbook.create_sheet(title=sheet_name)
    
    sheet = workbook[sheet_name]
    sheet.append(['Metric', 'Value'])

    for metric, value in zip(results['Metric'], results['Value']):
        sheet.append([metric, value])
    
    sheet.append([])
    sheet.append(['Confusion Matrix'])
    confusion_df = pd.DataFrame(confusion, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
    header = [''] + confusion_df.columns.tolist()
    sheet.append(header)
    for idx, row in confusion_df.iterrows():
        sheet.append([idx] + row.tolist())

    workbook.save(file_name)
    print(f'Classification results saved to {file_name}, sheet: {sheet_name}')

# Main function
def run_classification_metrics():
    # UI widgets
    directory_selector = widgets.Text(description='Directory Path:', layout=widgets.Layout(width='500px'))
    dir_confirm_button = widgets.Button(description='Confirm Directory', button_style='info')
    model_name_widget = widgets.Text(description='Model Name:', layout=widgets.Layout(width='500px'))
    dataset_type_widget = widgets.Dropdown(
        options=[('Select One', 'select_one'), ('Train', 'train'), ('Test', 'test')],
        description='Dataset Type:',
        layout=widgets.Layout(width='300px')
    )
    file_selector_pred = widgets.FileUpload(description='Upload y_pred File', multiple=False)
    uploaded_file_label = widgets.Label()
    threshold_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Threshold:', layout=widgets.Layout(width='500px'))
    run_button = widgets.Button(description='Run Metrics', button_style='success')
    
    # Containers
    result_output = widgets.Output()
    ui_output = widgets.Output()

    selected_directory = {'path': None}
    y_actual_files = {'train': None, 'test': None}
    uploaded_pred_data = {'filename': None, 'data': None}

    def on_dir_confirm_clicked(b):
        result_output.clear_output()
        path = directory_selector.value.strip()
        if not os.path.isdir(path):
            with result_output:
                print("Invalid directory. Please enter a valid path.")
            return

        selected_directory['path'] = path

        # Auto-select y_train and y_test
        try:
            for fname in os.listdir(path):
                if 'train' in fname.lower():
                    y_actual_files['train'] = os.path.join(path, fname)
                elif 'test' in fname.lower():
                    y_actual_files['test'] = os.path.join(path, fname)
        except Exception as e:
            with result_output:
                print(f"Error while reading files: {e}")

        with ui_output:
            ui_output.clear_output()
            display(widgets.VBox([
                model_name_widget,
                dataset_type_widget,
                file_selector_pred,
                uploaded_file_label,
                threshold_slider,
                run_button
            ]))

    def on_file_upload(change):
        uploaded_file_label.value = ""
        uploaded_pred_data['filename'] = None
        uploaded_pred_data['data'] = None

        try:
            if not file_selector_pred.value:
                uploaded_file_label.value = "Please upload a prediction file."
                return

            uploaded = next(iter(file_selector_pred.value.values()))
            uploaded_pred_data['filename'] = uploaded['metadata']['name']
            uploaded_pred_data['data'] = pd.read_csv(io.BytesIO(uploaded['content'])).iloc[:, 0]
            uploaded_file_label.value = f"Uploaded: {uploaded_pred_data['filename']}"
        except Exception as e:
            uploaded_file_label.value = f"Error reading file: {e}"

    def on_run_button_clicked(b):
        with result_output:
            if not selected_directory['path']:
                print("Please confirm directory path first.")
                return
            model_name = model_name_widget.value.strip()
            dataset_type = dataset_type_widget.value
            threshold = threshold_slider.value
            y_pred = uploaded_pred_data['data']

            if not model_name or dataset_type == 'select_one' or y_pred is None:
                print("Please enter a model name, select a dataset, and upload a valid prediction file.")
                return

            try:
                y_actual = pd.read_csv(y_actual_files[dataset_type]).iloc[:, 0]
                y_pred_bin = (y_pred >= threshold).astype(int)
                results, confusion = calculate_classification_metrics(y_actual, y_pred_bin)

                display(widgets.HTML(f"<h4 style='color:blue;'>Model: <b>{model_name}</b>, Dataset: <b>{dataset_type}</b>, Threshold: <b>{threshold:.2f}</b></h4>"))
                display(pd.DataFrame(results))
                display(pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1']))

                save_classification_results_to_excel(
                    selected_directory['path'],
                    model_name,
                    uploaded_pred_data['filename'],
                    results,
                    confusion
                )
            except Exception as e:
                print(f"An error occurred while processing the data: {e}")

    # Event bindings
    dir_confirm_button.on_click(on_dir_confirm_clicked)
    run_button.on_click(on_run_button_clicked)
    file_selector_pred.observe(on_file_upload, names='value')

    # Initial UI
    display(widgets.VBox([
        directory_selector,
        dir_confirm_button,
        ui_output,
        widgets.HTML("<hr><h3>Results:</h3>"),
        result_output
    ]))
